In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pathlib
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms

In [ ]:
filename='mpii_new.csv'
df = pd.read_csv(filename)
df

In [ ]:
#klasa koja ucitava dataset
class ImageDataset(Dataset):
    def __init__(self, heat_maps_dir, img_dir, transform, target_transform):
        self.heat_maps_dir = heat_maps_dir
        self.img_dir = img_dir
        self.transform = transform = transforms.Compose([
            transforms.Resize(transform),  # Resize the image to 128x128 pixels
            ])
        self.target_transform = transforms.Compose([
            transforms.Resize(target_transform),  # Resize the image to 128x128 pixels
            ])

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, os.listdir(self.img_dir)[idx])
        image = read_image(img_path)

        map_path = os.path.join(self.heat_maps_dir, os.listdir(self.heat_maps_dir)[idx])
        mapa = read_image(map_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            mapa = self.target_transform(mapa)
        return image, mapa


In [ ]:
#ucitavanje dataseta
dataset = ImageDataset('heat_maps_training', 'images_training', (368, 368), (368, 368))
slika, mapa = dataset[0]